In [7]:
%%time

import pandas as pd 
import scanpy as sc
import phenograph
import parc
from utag import utag
import os 
import numpy as np
import time
import anndata as ad
import scanpy.external as sce
import igraph
import umap
#import rapids_singlecell as rsc
#os.chdir("/home/abombin/Mawa/multiplex-analysis-web-apps")
os.chdir("/Users/bombina2/github/multiplex-analysis-web-apps")

CPU times: user 208 µs, sys: 299 µs, total: 507 µs
Wall time: 493 µs


In [2]:
# test with more than a million cells 
df = pd.read_csv("input/measurementsthymus-exported.csv")
list(df.columns)
meta = df.iloc[:, :4]
list(meta.columns)
mat = df.iloc[:, 4:]
list(mat.columns)
meta = meta.rename(columns={"Image": "Image ID_(standardized)", 
                            "Centroid X µm": "Centroid X (µm)_(standardized)", 
                            "Centroid Y µm": "Centroid Y (µm)_(standardized)"})

adata = ad.AnnData(mat)
adata.obs = meta
adata.layers["counts"] = adata.X.copy()

/Users/bombina2/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [4]:
%%time

communities, graph, Q = phenograph.cluster(adata.X, clustering_algo="louvian", k=30, 
                                               min_cluster_size=10, primary_metric="euclidean", 
                                               resolution_parameter=1, nn_method="kdtree",
                                               seed=42, n_iterations=-1, n_jobs=8)

Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 1390.8548481464386 seconds
Jaccard graph constructed in 45.790358781814575 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 1441.4985039234161 seconds
CPU times: user 3h 53min 13s, sys: 52.4 s, total: 3h 54min 5s
Wall time: 24min 1s


In [8]:
%%time
from umap.umap_ import nearest_neighbors
knn = nearest_neighbors(
                        adata.X,
                        n_neighbors=30,
                        metric='euclidean',
                        metric_kwds=None,
                        angular=False,
                        random_state=42
                        )

print(type(knn))
print(type(graph))

print(graph.shape[0])

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


<class 'tuple'>
<class 'scipy.sparse._coo.coo_matrix'>
1274717
CPU times: user 6min 3s, sys: 45.2 s, total: 6min 48s
Wall time: 52.1 s


In [10]:
print(knn)
print(graph)

(array([[      0,  870454,  161687, ...,  823722,   70499, 1107751],
       [      1,  603860,  601532, ...,  599847,  708798,  627706],
       [      2,   51771,  114050, ...,   62665,  319346,  261303],
       ...,
       [1274714, 1256833, 1273886, ...,  800905, 1249710, 1208227],
       [1274715, 1144173, 1251631, ...,  824023, 1271406, 1272150],
       [1274716, 1270200, 1251602, ..., 1207161, 1235460, 1241907]],
      dtype=int32), array([[ 0.      ,  9.505828, 10.602911, ..., 16.146563, 16.20209 ,
        16.390078],
       [ 0.      ,  8.727126,  8.901722, ..., 14.934456, 14.959417,
        15.012582],
       [ 0.      , 12.76424 , 14.644696, ..., 18.202799, 18.207096,
        18.32605 ],
       ...,
       [ 0.      , 38.888424, 39.50979 , ..., 52.573048, 53.151474,
        53.485657],
       [ 0.      , 33.1403  , 35.014458, ..., 52.587578, 52.62852 ,
        53.233257],
       [ 0.      , 38.68063 , 40.75001 , ..., 53.543903, 53.575253,
        53.93062 ]], dtype=float32), <

In [11]:
import numpy as np
from scipy.sparse import csr_matrix
 
def reformat_graph(graph, n_neighbors):
    # Convert the sparse matrix to the COO format
    graph_coo = graph.tocoo()
 
    # Initialize arrays for the indices and distances
    knn_indices = np.empty((graph.shape[0], n_neighbors), dtype=int)
    knn_dists = np.empty((graph.shape[0], n_neighbors))
 
    # Fill the arrays
    for i, (row, data) in enumerate(zip(graph_coo.row, graph_coo.data)):
        knn_indices[row, :] = graph_coo.col[i]
        knn_dists[row, :] = data
 
    return knn_indices, knn_dists
 
# Use the function
knn_indices, knn_dists = reformat_graph(graph=graph, n_neighbors=30)
precomputed_knn = (knn_indices, knn_dists)

In [12]:
print(precomputed_knn)

(array([[      0,       0,       0, ...,       0,       0,       0],
       [      0,       0,       0, ...,       0,       0,       0],
       [      0,       0,       0, ...,       0,       0,       0],
       ...,
       [1273886, 1273886, 1273886, ..., 1273886, 1273886, 1273886],
       [1274225, 1274225, 1274225, ..., 1274225, 1274225, 1274225],
       [1274374, 1274374, 1274374, ..., 1274374, 1274374, 1274374]]), array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.07142857, 0.07142857, 0.07142857, ..., 0.07142857, 0.07142857,
        0.07142857],
       [0.07692308, 0.07692308, 0.07692308, ..., 0.07692308, 0.07692308,
        0.07692308],
       [0.17647059, 0.17647059, 0.17647059, ..., 0.17647059, 0.17647059,
        0.17647059]]))

In [28]:
graph_matrix = graph.tocoo
X_df = ad.AnnData.to_df(adata)

print(len(adata.X))
print(len(graph_matrix))

In [14]:
%%time
knn_umap = umap.UMAP(n_neighbors = 30, n_components=2, metric='euclidean', 
                     n_jobs = 8, precomputed_knn = knn).fit_transform(adata.X)

CPU times: user 1h 53min 26s, sys: 1h 5min 39s, total: 2h 59min 6s
Wall time: 1h 2min 58s


In [15]:
import scanpy as sc
import phenograph
import numpy as np
from sklearn.preprocessing import normalize
from umap import UMAP
 

# Reformat the Phenograph graph to a kNN representation
def reformat_phenograph_graph(graph, n_neighbors):
    N = graph.distances.shape[0]
    knn_indices = np.zeros((N, n_neighbors), dtype=int)
    knn_dists = np.zeros((N, n_neighbors))
 
    for i in range(N):
        neighbors = sorted(zip(graph.distances[i], graph.transitions[i]), key=lambda x: x[0])[:n_neighbors]
        neighbor_indices, neighbor_dists = zip(*neighbors)
        knn_indices[i] = neighbor_indices
        knn_dists[i] = neighbor_dists
 
    return knn_indices, knn_dists
 
# Get the kNN indices and distances from the Phenograph graph
n_neighbors = 30
knn_indices, knn_dists = reformat_phenograph_graph(graph, n_neighbors)
precomputed_knn_1 = (knn_indices, knn_dists)
print(precomputed_knn_1) 


AttributeError: 'coo_matrix' object has no attribute 'distances'

In [3]:
%%time
cur_umap = umap.UMAP(n_neighbors = 30, n_components=2, 
                     n_jobs = 8).fit_transform(adata.X)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


CPU times: user 1h 58min 30s, sys: 1h 4min 18s, total: 3h 2min 49s
Wall time: 54min 58s


In [11]:
%%time
# test with more than a million cells 
df = pd.read_csv("input/measurementsthymus-exported.csv")
list(df.columns)
meta = df.iloc[:, :4]
list(meta.columns)
mat = df.iloc[:, 4:]
list(mat.columns)
meta = meta.rename(columns={"Image": "Image ID_(standardized)", 
                            "Centroid X µm": "Centroid X (µm)_(standardized)", 
                            "Centroid Y µm": "Centroid Y (µm)_(standardized)"})

adata = ad.AnnData(mat)
adata.obs = meta
#adata.raw = adata
adata.layers["counts"] = adata.X.copy()

sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.tl.pca(adata, n_comps=10)

/Users/bombina2/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


CPU times: user 7.26 s, sys: 6.17 s, total: 13.4 s
Wall time: 3.84 s


In [14]:
%%time
sc.pp.neighbors(adata, n_neighbors=30, metric="euclidean", n_pcs=0)

CPU times: user 4min 19s, sys: 6.43 s, total: 4min 26s
Wall time: 4min 24s


In [ ]:
%%time
sc.pp.neighbors(adata, n_neighbors=30, metric="euclidean", n_pcs=10)

In [3]:
%%time
import umap
cur_umap_pca = umap.UMAP(n_neighbors=30, n_components=2, n_jobs=8).fit_transform(adata.obsm['X_pca'])

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


CPU times: user 1h 58min 18s, sys: 1h 5min 7s, total: 3h 3min 25s
Wall time: 55min 26s


In [4]:
%%time
cur_umap = umap.UMAP(n_neighbors = 30, n_components=2, 
                     n_jobs = 8).fit_transform(adata.X)

CPU times: user 1h 59min 17s, sys: 1h 6min 2s, total: 3h 5min 20s
Wall time: 58min 5s


In [6]:
%%time

communities, graph, Q = phenograph.cluster(adata.obsm['X_pca'], clustering_algo="louvian", k=30, 
                                               min_cluster_size=10, primary_metric="euclidean", 
                                               resolution_parameter=1, nn_method="kdtree",
                                               seed=42, n_iterations=-1, n_jobs=8)

Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 296.0105941295624 seconds
Jaccard graph constructed in 46.47930908203125 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 347.2421510219574 seconds
CPU times: user 37min 7s, sys: 17.4 s, total: 37min 24s
Wall time: 5min 47s


In [3]:
%%time
from sklearn.preprocessing import StandardScaler
# test with more than a million cells 
df = pd.read_csv("input/measurementsthymus-exported.csv")
list(df.columns)
meta = df.iloc[:, :4]
list(meta.columns)
mat = df.iloc[:, 4:]
list(mat.columns)
meta = meta.rename(columns={"Image": "Image ID_(standardized)", 
                            "Centroid X µm": "Centroid X (µm)_(standardized)", 
                            "Centroid Y µm": "Centroid Y (µm)_(standardized)"})

adata = ad.AnnData(mat)
adata.obs = meta
#adata.raw = adata
adata.layers["counts"] = adata.X.copy()

sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.tl.pca(adata, n_comps=10)

scaled_data = StandardScaler().fit_transform(adata.X)

/Users/bombina2/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


CPU times: user 7.65 s, sys: 7.34 s, total: 15 s
Wall time: 4.14 s


In [4]:
%%time
cur_umap = umap.UMAP(n_neighbors = 30, n_components=2, 
                     n_jobs = 8).fit_transform(scaled_data)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


CPU times: user 2h 52s, sys: 1h 9min 33s, total: 3h 10min 25s
Wall time: 57min 31s


In [8]:
%%time
from sklearn.preprocessing import StandardScaler
# test with more than a million cells 
df = pd.read_csv("input/measurementsthymus-exported.csv")
list(df.columns)
meta = df.iloc[:, :4]
list(meta.columns)
mat = df.iloc[:, 4:]
list(mat.columns)
meta = meta.rename(columns={"Image": "Image ID_(standardized)", 
                            "Centroid X µm": "Centroid X (µm)_(standardized)", 
                            "Centroid Y µm": "Centroid Y (µm)_(standardized)"})

adata = ad.AnnData(mat)
adata.obs = meta
#adata.raw = adata
adata.layers["counts"] = adata.X.copy()

#sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, n_comps=10)

/Users/bombina2/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


CPU times: user 6.74 s, sys: 6.59 s, total: 13.3 s
Wall time: 4.04 s


In [3]:
%%time
import umap
cur_umap_pca = umap.UMAP(n_neighbors=30, n_components=2, n_jobs=8).fit_transform(adata.obsm['X_pca'])

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


CPU times: user 1h 58min 10s, sys: 1h 7min 4s, total: 3h 5min 14s
Wall time: 55min 57s


In [9]:
%%time
import umap
cur_umap_pca_cos = umap.UMAP(n_neighbors=30, n_components=2, n_jobs=8, metric="cosine").fit_transform(adata.obsm['X_pca'])

CPU times: user 2h 56min 36s, sys: 1h 2min 10s, total: 3h 58min 47s
Wall time: 1h 8min 52s


In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(adata.obsm['X_pca'])


In [5]:
%%time
import umap
cur_umap_pca = umap.UMAP(n_neighbors=30, n_components=2, n_jobs=8).fit_transform(data_scaled)

CPU times: user 2h 56min 16s, sys: 1h 7min 48s, total: 4h 4min 4s
Wall time: 1h 3min 14s


In [2]:
%%time

# test with more than a million cells 
df = pd.read_csv("input/measurementsthymus-exported.csv")
list(df.columns)
meta = df.iloc[:, :4]
list(meta.columns)
mat = df.iloc[:, 4:]
list(mat.columns)
meta = meta.rename(columns={"Image": "Image ID_(standardized)", 
                            "Centroid X µm": "Centroid X (µm)_(standardized)", 
                            "Centroid Y µm": "Centroid Y (µm)_(standardized)"})

adata = ad.AnnData(mat)
adata.obs = meta
#adata.raw = adata
adata.layers["counts"] = adata.X.copy()

#sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
#print(adata.X)
sc.pp.scale(adata)
#print(adata.X)
sc.pp.pca(adata, n_comps=10)

/Users/bombina2/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


CPU times: user 8.2 s, sys: 6.11 s, total: 14.3 s
Wall time: 4.79 s


In [3]:
%%time
sc.pp.neighbors(adata, n_neighbors=10, metric='euclidean', n_pcs=10)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


CPU times: user 2min 48s, sys: 18.3 s, total: 3min 7s
Wall time: 2min 55s


In [5]:
%%time
sc.tl.tsne(adata, n_pcs=10, n_jobs=8)


/Users/bombina2/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/scanpy/tools/_tsne.py:124: UserWarning: In previous versions of scanpy, calling tsne with n_jobs > 1 would use MulticoreTSNE. Now this uses the scikit-learn version of TSNE by default. If you'd like the old behaviour (which is deprecated), pass 'use_fast_tsne=True'. Note, MulticoreTSNE is not actually faster anymore.
  warnings.warn(


CPU times: user 12h 21min 16s, sys: 23min 42s, total: 12h 44min 59s
Wall time: 2h 13min 22s


In [6]:
%%time
import MulticoreTSNE
sc.tl.tsne(adata, n_pcs=10, n_jobs=8, use_fast_tsne=True)

RuntimeError: Cannot find/open tsne_multicore shared library